In [11]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from src.PDLSSVM import PDLSSVM
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix, accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=1234)

# iris dataset

In [16]:
from tqdm.notebook import tqdm

iris = load_iris()
X = iris['data'][:100]
y = iris['target'][:100]
y[:50] = -1

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

rho_search_space = [1e-6, 1e-5, 1e-4, 1e-3]
c_search_space   = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c1_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c2_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]

# rho_search_space = [1e-4]
# c_search_space   = [1e+5]
# c1_search_space  = [1e-7]
# c2_search_space  = [1e-8]

results_dict = {"rho":[], "c": [], "c1": [], "c2": [], "Acc": []}

for rho in tqdm(rho_search_space):
    for c in c_search_space:
        for c1 in c1_search_space:
            for c2 in c2_search_space:
                cv_acc = []
                for train_index, val_index in kf.split(X):
                    X_train, X_val = X[train_index], X[val_index]
                    y_train, y_val = y[train_index], y[val_index]
                
                    clf = PDLSSVM(rho, c, c1, c2)
                    clf.fit(X_train, y_train)
                    pred, _, _ = clf.predict(X_val)
                    acc = accuracy_score(y_val, pred)
                    cv_acc.append(acc)
                
                cv_acc = np.mean(cv_acc)
                results_dict['rho'].append(rho)
                results_dict['c'].append(c)
                results_dict['c1'].append(c1)
                results_dict['c2'].append(c2)
                results_dict['Acc'].append(cv_acc)
                
results_df = pd.DataFrame.from_dict(results_dict)
results_df

  0%|          | 0/4 [00:00<?, ?it/s]

,rho,c,c1,c2,Acc
0,0.000001,1.000000e-08,1.000000e-08,1.000000e-08,0.8375
1,0.000001,1.000000e-08,1.000000e-08,1.000000e-07,0.8375
2,0.000001,1.000000e-08,1.000000e-08,1.000000e-06,0.8375
3,0.000001,1.000000e-08,1.000000e-08,1.000000e-05,0.8375
4,0.000001,1.000000e-08,1.000000e-08,1.000000e-04,0.8375
...,...,...,...,...,...
16379,0.001000,1.000000e+08,1.000000e+08,1.000000e+04,1.0000
16380,0.001000,1.000000e+08,1.000000e+08,1.000000e+05,1.0000
16381,0.001000,1.000000e+08,1.000000e+08,1.000000e+06,1.0000
16382,0.001000,1.000000e+08,1.000000e+08,1.000000e+07,1.0000


In [17]:
results_df.sort_values('Acc', ascending=False)

,rho,c,c1,c2,Acc
16383,0.00100,1.000000e+08,1.000000e+08,1.000000e+08,1.0000
12223,0.00010,1.000000e+08,1.000000e+04,1.000000e+08,1.0000
6334,0.00001,1.000000e+01,1.000000e+04,1.000000e+07,1.0000
6335,0.00001,1.000000e+01,1.000000e+04,1.000000e+08,1.0000
12227,0.00010,1.000000e+08,1.000000e+05,1.000000e-05,1.0000
...,...,...,...,...,...
8645,0.00010,1.000000e-07,1.000000e+05,1.000000e-03,0.4625
8646,0.00010,1.000000e-07,1.000000e+05,1.000000e-02,0.4625
8647,0.00010,1.000000e-07,1.000000e+05,1.000000e-01,0.4625
8648,0.00010,1.000000e-07,1.000000e+05,1.000000e+01,0.4625


In [18]:
#PDLSSVM
rho = 1e-3
c   = 1e+8
c1  = 1e-8
c2  = 1e-8

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

Training Done
Total Running Time: 0.0494
Prediction Done
Total Running Time: 0.0004


array([[ 7,  0],
       [ 0, 13]])

In [21]:
# LSSVM
rho = 1e-4
c   = 1e+5
c1  = 0
c2  = 0

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

Training Done
Total Running Time: 0.0743
Prediction Done
Total Running Time: 0.0003


1.0

In [22]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X, y)
pred = clf.predict(X_test)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

1.0

# sonar dataset

In [23]:
df = pd.read_table("data/sonar.all-data", sep=",", header=None)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].valuesa

y[np.isin(y, "R")] = -1
y[np.isin(y, "M")] = 1
y = np.expand_dims(y, -1).astype('int64')

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=1234, stratify=y)

In [24]:
rho_search_space = [1e-6, 1e-5, 1e-4, 1e-3]
c_search_space   = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c1_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c2_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]

results_dict = {"rho":[], "c": [], "c1": [], "c2": [], "Acc": []}
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

for rho in tqdm(rho_search_space):
    for c in c_search_space:
        for c1 in c1_search_space:
            for c2 in c2_search_space:
                cv_acc = []
                for train_index, val_index in kf.split(X):
                    X_train, X_val = X[train_index], X[val_index]
                    y_train, y_val = y[train_index], y[val_index]
                
                    clf = PDLSSVM(rho, c, c1, c2)
                    clf.fit(X_train, y_train)
                    pred, _, _ = clf.predict(X_val)
                    pred.astype('int64')
                    
                    acc = accuracy_score(y_val, pred)1
                    cv_acc.append(acc)
                
                cv_acc = np.mean(cv_acc)
                results_dict['rho'].append(rho)
                results_dict['c'].append(c)
                results_dict['c1'].append(c1)
                results_dict['c2'].append(c2)
                results_dict['Acc'].append(cv_acc)
                
results_df = pd.DataFrame.from_dict(results_dict)
results_df

  0%|          | 0/4 [00:00<?, ?it/s]

,rho,c,c1,c2,Acc
0,0.000001,1.000000e-08,1.000000e-08,1.000000e-08,0.541176
1,0.000001,1.000000e-08,1.000000e-08,1.000000e-07,0.541176
2,0.000001,1.000000e-08,1.000000e-08,1.000000e-06,0.541176
3,0.000001,1.000000e-08,1.000000e-08,1.000000e-05,0.541176
4,0.000001,1.000000e-08,1.000000e-08,1.000000e-04,0.541176
...,...,...,...,...,...
16379,0.001000,1.000000e+08,1.000000e+08,1.000000e+04,0.716578
16380,0.001000,1.000000e+08,1.000000e+08,1.000000e+05,0.716578
16381,0.001000,1.000000e+08,1.000000e+08,1.000000e+06,0.716578
16382,0.001000,1.000000e+08,1.000000e+08,1.000000e+07,0.716578


In [26]:
results_df.sort_values('Acc', ascending=False)

,rho,c,c1,c2,Acc
7078,0.00001,10000.0,1000.0,1.000000e-02,0.770766
7387,0.00001,100000.0,1000000.0,1.000000e+04,0.764884
7385,0.00001,100000.0,1000000.0,1.000000e+02,0.764884
7384,0.00001,100000.0,1000000.0,1.000000e+01,0.764884
7375,0.00001,100000.0,100000.0,1.000000e+08,0.764884
...,...,...,...,...,...
7376,0.00001,100000.0,1000000.0,1.000000e-08,0.451337
7649,0.00001,1000000.0,10000000.0,1.000000e-07,0.451337
7648,0.00001,1000000.0,10000000.0,1.000000e-08,0.451337
7415,0.00001,100000.0,100000000.0,1.000000e-01,0.451337


In [27]:
#PDLSSVM
rho = 1e-5
c   = 1e+4
c1  = 1e+3
c2  = 1e-2

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

Training Done
Total Running Time: 0.1258
Prediction Done
Total Running Time: 0.0002


0.7142857142857143

In [28]:
# LSSVM
rho = 1e-5
c   = 1e+4
c1  = 0
c2  = 0

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

Training Done
Total Running Time: 0.1165
Prediction Done
Total Running Time: 0.0002


0.7619047619047619

In [29]:
clf = SVC()
clf.fit(X, y)
pred = clf.predict(X_test)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

/Users/jonghyun/miniforge3/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


0.8333333333333334

# dbworld dataset

In [13]:
data = np.load("data/dbworld.npy")
X = data[:, :-1]
y = data[:, -1]

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=1234, stratify=y)

In [16]:
rho_search_space = [1e-6, 1e-5, 1e-4, 1e-3]
c_search_space   = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c1_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c2_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]

results_dict = {"rho":[], "c": [], "c1": [], "c2": [], "Acc": []}
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

for rho in tqdm(rho_search_space):
    for c in c_search_space:
        for c1 in c1_search_space:
            for c2 in c2_search_space:
                cv_acc = []
                for train_index, val_index in kf.split(X):
                    X_train, X_val = X[train_index], X[val_index]
                    y_train, y_val = y[train_index], y[val_index]
                
                    clf = PDLSSVM(rho, c, c1, c2)
                    clf.fit(X_train, y_train)
                    pred, _, _ = clf.predict(X_val)
                    pred.astype('int64')
                    
                    acc = accuracy_score(y_val, pred)
                    cv_acc.append(acc)
                
                cv_acc = np.mean(cv_acc)
                results_dict['rho'].append(rho)
                results_dict['c'].append(c)
                results_dict['c1'].append(c1)
                results_dict['c2'].append(c2)
                results_dict['Acc'].append(cv_acc)
                
results_df = pd.DataFrame.from_dict(results_dict)
results_df

,rho,c,c1,c2,Acc
0,0.000001,1.000000e-08,1.000000e-08,1.000000e-08,0.843636
1,0.000001,1.000000e-08,1.000000e-08,1.000000e-07,0.863636
2,0.000001,1.000000e-08,1.000000e-08,1.000000e-06,0.863636
3,0.000001,1.000000e-08,1.000000e-08,1.000000e-05,0.863636
4,0.000001,1.000000e-08,1.000000e-08,1.000000e-04,0.843636
...,...,...,...,...,...
16379,0.001000,1.000000e+08,1.000000e+08,1.000000e+04,0.843636
16380,0.001000,1.000000e+08,1.000000e+08,1.000000e+05,0.843636
16381,0.001000,1.000000e+08,1.000000e+08,1.000000e+06,0.843636
16382,0.001000,1.000000e+08,1.000000e+08,1.000000e+07,0.843636


In [17]:
results_df.sort_values('Acc', ascending=False)

,rho,c,c1,c2,Acc
8192,0.0001,1.000000e-08,1.000000e-08,1.000000e-08,0.863636
13610,0.0010,1.000000e-03,1.000000e-06,1.000000e+03,0.863636
13630,0.0010,1.000000e-03,1.000000e-05,1.000000e+07,0.863636
13629,0.0010,1.000000e-03,1.000000e-05,1.000000e+06,0.863636
13628,0.0010,1.000000e-03,1.000000e-05,1.000000e+05,0.863636
...,...,...,...,...,...
14583,0.0010,1.000000e+01,1.000000e+08,1.000000e-01,0.447273
14582,0.0010,1.000000e+01,1.000000e+08,1.000000e-02,0.447273
14581,0.0010,1.000000e+01,1.000000e+08,1.000000e-03,0.447273
14580,0.0010,1.000000e+01,1.000000e+08,1.000000e-04,0.447273


In [22]:
#PDLSSVM
rho = 1e-3
c   = 1e-3
c1  = 1e-5
c2  = 1e-5

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

Training Done
Total Running Time: 0.123
Prediction Done
Total Running Time: 0.001


0.9230769230769231

In [19]:
#LSSVM
rho = 1e-3
c   = 1e-3
c1  = 0
c2  = 0

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

Training Done
Total Running Time: 0.122
Prediction Done
Total Running Time: 0.001


0.9230769230769231

In [21]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X, y)
pred = clf.predict(X_test)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)

1.0

# heart attack

In [30]:
data = pd.read_csv("data/heart_attack.csv")
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=1234, shuffle=True, stratify=y)

In [33]:
rho_search_space = [1e-6, 1e-5, 1e-4, 1e-3]
c_search_space   = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c1_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]
c2_search_space  = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1e+1, 1e+2, 1e+3, 1e+4, 1e+5, 1e+6, 1e+7, 1e+8]

results_dict = {"rho":[], "c": [], "c1": [], "c2": [], "Acc": []}
kf = KFold(n_splits=5, shuffle=True, random_state=1234)

for rho in tqdm(rho_search_space):
    for c in c_search_space:
        for c1 in c1_search_space:
            for c2 in c2_search_space:
                cv_acc = []
                for train_index, val_index in kf.split(X):
                    X_train, X_val = X[train_index], X[val_index]
                    y_train, y_val = y[train_index], y[val_index]
                
                    clf = PDLSSVM(rho, c, c1, c2)
                    clf.fit(X_train, y_train)
                    pred, _, _ = clf.predict(X_val)
                    pred.astype('int64')
                    
                    acc = accuracy_score(y_val, pred)
                    cv_acc.append(acc)
                
                cv_acc = np.mean(cv_acc)
                results_dict['rho'].append(rho)
                results_dict['c'].append(c)
                results_dict['c1'].append(c1)
                results_dict['c2'].append(c2)
                results_dict['Acc'].append(cv_acc)
                
results_df = pd.DataFrame.from_dict(results_dict)
results_df

KeyboardInterrupt: 

In [ ]:
results_df.sort_values('Acc', ascending=False)

In [ ]:
#PDLSSVM
rho = 1e-3
c   = 1e-3
c1  = 1e-5
c2  = 1e-5

clf = PDLSSVM(rho, c, c1, c2)
clf.fit(X, y, verbose=True)
pred, _, _ = clf.predict(X_test, verbose=True)
# confusion_matrix(y_test, pred)
accuracy_score(y_test, pred)